### Install dependencies

In [ ]:
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent wget torf && apt install python3-libtorrent

### Download torrent

In [ ]:
import libtorrent as lt
import time
import datetime
import os
import shutil
from google.colab import files
import wget
import torf

def parse_selection(selection):
    if selection.startswith("-"):
        mode = "exclude"
        selection = selection[1:]
    else:
        mode = "include"
    ranges = (r.split("-") for r in selection.split(","))
    numbers = []
    for r in ranges:
        if len(r) == 1:
            numbers.append(int(r[0]))
        else:
            numbers.extend(range(int(r[0]), int(r[1]) + 1))
    numbers = [x - 1 for x in numbers]
    return mode, numbers

# Initialize session
ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': '/content/Torrent/',
    'storage_mode': lt.storage_mode_t(2),
}
ses.start_dht()

# Input Torrent or Magnet link
link = input("Input Torrent file link or Magnet link and Press Enter: ")

# Handling .torrent files
if link.endswith('.torrent'):
    if os.path.exists('torrent.torrent'):
        os.remove('torrent.torrent')
    wget.download(link, 'torrent.torrent')
    t = torf.Torrent.read('torrent.torrent')
    link = str(t.magnet())

# Add torrent
handle = lt.add_magnet_uri(ses, link, params)
handle.set_sequential_download(0)

print(datetime.datetime.now())
print('Downloading Metadata...')

while not handle.has_metadata():
    time.sleep(1)

torrent_info = handle.get_torrent_info()
print('Got Metadata, Starting Torrent Download...')
print("Starting", torrent_info.name())

print("\nSelect the files you want to download (you can use ranges and exclusions):")
for i, file in enumerate(torrent_info.files()):
    print(f"{i+1}. {file.path} ({file.size/1024/1024:.2f} MB)")

print("\nExamples of how to select files:")
print("1. Enter '1-3' to select files 1 through 3.")
print("2. Enter '2,4,6' to select files 2, 4, and 6 specifically.")
print("3. Enter '-5' to exclude file 5 from downloading.")
print("4. Enter '1-3,7' to select files 1 through 3 and file 7.")
print("5. Enter '-2-4' to exclude files 2 through 4 from downloading.")
print("Enter 'done' when you are finished selecting.\n")

all_files = set(range(torrent_info.num_files()))
selected_files = set()
excluded_files = set()

while True:
    selection = input("Enter your selection, or 'done': ").strip()
    if selection.lower() == "done":
        if not selected_files and not excluded_files:
            selected_files = all_files
        break
    mode, numbers = parse_selection(selection)
    if mode == "exclude":
        excluded_files.update(numbers)
    else:
        if selected_files == all_files:
            selected_files.clear()
        selected_files.update(numbers)

final_selection = (selected_files if not excluded_files else selected_files - excluded_files) or all_files

for i in all_files:
    priority = 1 if i in final_selection else 0
    handle.file_priority(i, priority)

# Adjust the download monitoring loop
print(f"\nDownloading {len(final_selection)} files...")
begin = time.time()

while True:
    s = handle.status()
    print(f'{s.progress * 100:.2f}% complete (down: {s.download_rate / 1000:.1f} kb/s up: {s.upload_rate / 1000:.1f} kB/s peers: {s.num_peers}) {["queued", "checking", "downloading metadata", "downloading", "finished", "seeding", "allocating"][s.state]}')
    
    # Check if the download is effectively complete
    if s.state == lt.torrent_status.seeding or (s.progress >= 1.0 and s.download_rate < 1000):
        print("Download effectively complete.")
        break
    time.sleep(5)

end = time.time()
print(f"{torrent_info.name()} COMPLETE")
print(f"Elapsed Time: {int((end - begin) // 60)} min : {int((end - begin) % 60)} sec")
print(datetime.datetime.now())


# Compress the downloaded content for easy download
downloaded_dir = '/content/Torrent/'  # Adjust if needed
zip_file_name = '/content/torrent_download.zip'
shutil.make_archive(zip_file_name.rsplit('.', 1)[0], 'zip', downloaded_dir)
print(f"Compressed downloaded files into {zip_file_name}")

# Trigger the download
files.download(zip_file_name)


### Zip the files

In [ ]:
print('Zipping started...')

ziph = zipfile.ZipFile('Downloaded-Torrent.zip', 'w', zipfile.ZIP_DEFLATED)
path = 'content/Torrent/'
for root, dirs, files in os.walk(path):
    for file in files:
        ziph.write(os.path.join(root, file),
                   os.path.relpath(os.path.join(root, file),
                                   os.path.join(path, '..')))
ziph.close()

print('\nALL DONE!')